<a href="https://colab.research.google.com/github/mertege/Thesis_Experiments/blob/main/3_Spectrograms_CI4R_Datasets_Multi_Class_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Burada CI4R dataseti ile Multi Class Classification yapıldı.
# Image-based Architecure
# Farklı frekansların inputları 3 farklı path'den geçti ve sonra concat edildi.
# 3-blocks are used. Resolution 128x128
# It was run 5-times. Results are shared on below:
# Min test accuracy is 0.82, Max test accuracy is 0.84, Mean test accuracy is 0.83, std of test accuracy is 0.01.
# Time elapsed during training time: 70.26, sec
# Time elapsed during test time: 0.27, sec

In [4]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization, Input, Conv2D, MaxPooling2D, Concatenate, GRU
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K 
import gc
import os

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# 77 GHz
bending_77GHz                 = np.load('/content/drive/MyDrive/bending_77Ghz.npy')
label_bending_77GHz           = np.zeros((bending_77GHz.shape[0],1))
label_bending_77GHz[:]        = 1
crawling_77GHz                = np.load('/content/drive/MyDrive/crawling_77Ghz.npy')
label_crawling_77GHz          = np.zeros((crawling_77GHz.shape[0],1))
label_crawling_77GHz[:]       = 2
kneeling_77GHz                = np.load('/content/drive/MyDrive/kneeling_77Ghz.npy')
label_kneeling_77GHz          = np.zeros((kneeling_77GHz.shape[0],1))
label_kneeling_77GHz[:]       = 3
limping_with_RL_Stiff_77GHz   = np.load('/content/drive/MyDrive/limping_with_RL_Stiff_77Ghz.npy')
label_limping_with_RL_Stiff_77GHz           = np.zeros((limping_with_RL_Stiff_77GHz.shape[0],1))
label_limping_with_RL_Stiff_77GHz[:]        = 4
picking_up_an_object_77GHz    = np.load('/content/drive/MyDrive/picking_up_an_object_77Ghz.npy')
label_picking_up_an_object_77GHz          = np.zeros((picking_up_an_object_77GHz.shape[0],1))
label_picking_up_an_object_77GHz[:]       = 5
scissors_gait_77GHz           = np.load('/content/drive/MyDrive/scissors_gait_77Ghz.npy')
label_scissors_gait_77GHz          = np.zeros((scissors_gait_77GHz.shape[0],1))
label_scissors_gait_77GHz[:]       = 6
short_steps_77GHz             = np.load('/content/drive/MyDrive/short_steps_77Ghz.npy')
label_short_steps_77GHz          = np.zeros((short_steps_77GHz.shape[0],1))
label_short_steps_77GHz[:]       = 7
sitting_77GHz                 = np.load('/content/drive/MyDrive/sitting_77Ghz.npy')
label_sitting_77GHz          = np.zeros((sitting_77GHz.shape[0],1))
label_sitting_77GHz[:]       = 8
walking_away_from_Radar_77GHz = np.load('/content/drive/MyDrive/walking_away_from_Radar_77Ghz.npy')
label_walking_away_from_Radar_77GHz         = np.zeros((walking_away_from_Radar_77GHz.shape[0],1))
label_walking_away_from_Radar_77GHz[:]       = 9
Walking_on_both_toes_77GHz    = np.load('/content/drive/MyDrive/Walking_on_both_toes_77Ghz.npy')
label_Walking_on_both_toes_77GHz         = np.zeros((Walking_on_both_toes_77GHz.shape[0],1))
label_Walking_on_both_toes_77GHz[:]       = 10
Walking_towards_radar_77GHz   = np.load('/content/drive/MyDrive/Walking_towards_radar_77Ghz.npy')
label_Walking_towards_radar_77GHz          = np.zeros((Walking_towards_radar_77GHz.shape[0],1))
label_Walking_towards_radar_77GHz[:]       = 11
# 24 GHz
bending_24GHz                 = np.load('/content/drive/MyDrive/bending_24GHz.npy')
label_bending_24GHz           = np.zeros((bending_24GHz.shape[0],1))
label_bending_24GHz[:]        = 1
crawling_24GHz                = np.load('/content/drive/MyDrive/crawling_24GHz.npy')
label_crawling_24GHz          = np.zeros((crawling_24GHz.shape[0],1))
label_crawling_24GHz[:]       = 2
kneeling_24GHz                = np.load('/content/drive/MyDrive/kneeling_24GHz.npy')
label_kneeling_24GHz          = np.zeros((kneeling_24GHz.shape[0],1))
label_kneeling_24GHz[:]       = 3
limping_with_RL_Stiff_24GHz   = np.load('/content/drive/MyDrive/limping with right stiff_24GHz.npy')
label_limping_with_RL_Stiff_24GHz           = np.zeros((limping_with_RL_Stiff_24GHz.shape[0],1))
label_limping_with_RL_Stiff_24GHz[:]        = 4
picking_up_an_object_24GHz    = np.load('/content/drive/MyDrive/picking up object_24GHz.npy')
label_picking_up_an_object_24GHz          = np.zeros((picking_up_an_object_24GHz.shape[0],1))
label_picking_up_an_object_24GHz[:]       = 5
scissors_gait_24GHz           = np.load('/content/drive/MyDrive/scissors gait_24GHz.npy')
label_scissors_gait_24GHz          = np.zeros((scissors_gait_24GHz.shape[0],1))
label_scissors_gait_24GHz[:]       = 6
short_steps_24GHz             = np.load('/content/drive/MyDrive/short steps_24GHz.npy')
label_short_steps_24GHz          = np.zeros((short_steps_24GHz.shape[0],1))
label_short_steps_24GHz[:]       = 7
sitting_24GHz                 = np.load('/content/drive/MyDrive/sitting_24GHz.npy')
label_sitting_24GHz          = np.zeros((sitting_24GHz.shape[0],1))
label_sitting_24GHz[:]       = 8
walking_away_from_Radar_24GHz = np.load('/content/drive/MyDrive/walking away from radar_24GHz.npy')
label_walking_away_from_Radar_24GHz         = np.zeros((walking_away_from_Radar_24GHz.shape[0],1))
label_walking_away_from_Radar_24GHz[:]       = 9
Walking_on_both_toes_24GHz    = np.load('/content/drive/MyDrive/walking on toes both_24GHz.npy')
label_Walking_on_both_toes_24GHz         = np.zeros((Walking_on_both_toes_24GHz.shape[0],1))
label_Walking_on_both_toes_24GHz[:]       = 10
Walking_towards_radar_24GHz   = np.load('/content/drive/MyDrive/walking towards radar_24GHz.npy')
label_Walking_towards_radar_24GHz          = np.zeros((Walking_towards_radar_24GHz.shape[0],1))
label_Walking_towards_radar_24GHz[:]       = 11
# Xethru
bending_Xethru                 = np.load('/content/drive/MyDrive/bending_Xethru.npy')
label_bending_Xethru           = np.zeros((bending_Xethru.shape[0],1))
label_bending_Xethru[:]        = 1
crawling_Xethru                = np.load('/content/drive/MyDrive/crawling_Xethru.npy')
label_crawling_Xethru          = np.zeros((crawling_Xethru.shape[0],1))
label_crawling_Xethru[:]       = 2
kneeling_Xethru                = np.load('/content/drive/MyDrive/kneeling_Xethru.npy')
label_kneeling_Xethru          = np.zeros((kneeling_Xethru.shape[0],1))
label_kneeling_Xethru[:]       = 3
limping_with_RL_Stiff_Xethru   = np.load('/content/drive/MyDrive/limping with right leg stiff_Xethru.npy')
label_limping_with_RL_Stiff_Xethru           = np.zeros((limping_with_RL_Stiff_Xethru.shape[0],1))
label_limping_with_RL_Stiff_Xethru[:]        = 4
picking_up_an_object_Xethru    = np.load('/content/drive/MyDrive/ picking up an object_Xethru.npy')
label_picking_up_an_object_Xethru          = np.zeros((picking_up_an_object_Xethru.shape[0],1))
label_picking_up_an_object_Xethru[:]       = 5
scissors_gait_Xethru           = np.load('/content/drive/MyDrive/scissor Gait_Xethru.npy')
label_scissors_gait_Xethru          = np.zeros((scissors_gait_Xethru.shape[0],1))
label_scissors_gait_Xethru[:]       = 6
short_steps_Xethru             = np.load('/content/drive/MyDrive/short Step_Xethru.npy')
label_short_steps_Xethru          = np.zeros((short_steps_Xethru.shape[0],1))
label_short_steps_Xethru[:]       = 7
sitting_Xethru                 = np.load('/content/drive/MyDrive/sitting_Xethru.npy')
label_sitting_Xethru          = np.zeros((sitting_Xethru.shape[0],1))
label_sitting_Xethru[:]       = 8
walking_away_from_Radar_Xethru = np.load('/content/drive/MyDrive/Walking away from radar_Xethru.npy')
label_walking_away_from_Radar_Xethru         = np.zeros((walking_away_from_Radar_Xethru.shape[0],1))
label_walking_away_from_Radar_Xethru[:]       = 9
Walking_on_both_toes_Xethru    = np.load('/content/drive/MyDrive/walking on both toes_Xethru.npy')
label_Walking_on_both_toes_Xethru         = np.zeros((Walking_on_both_toes_Xethru.shape[0],1))
label_Walking_on_both_toes_Xethru[:]       = 10
Walking_towards_radar_Xethru   = np.load('/content/drive/MyDrive/Walking towards the radar_Xethru.npy')
label_Walking_towards_radar_Xethru          = np.zeros((Walking_towards_radar_Xethru.shape[0],1))
label_Walking_towards_radar_Xethru[:]       = 11

In [7]:
min_length = min(label_bending_77GHz.shape[0],label_bending_24GHz.shape[0],label_bending_Xethru.shape[0])
label_concat = label_bending_77GHz[:min_length]
bending_77GHz = bending_77GHz[:min_length,:,:,:]
bending_24GHz = bending_24GHz[:min_length,:,:,:]
bending_Xethru = bending_Xethru[:min_length,:,:,:]

min_length = min(label_crawling_77GHz.shape[0],label_crawling_24GHz.shape[0],label_crawling_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_crawling_77GHz[:min_length]),axis=0)  
crawling_77GHz = crawling_77GHz[:min_length,:,:,:]
crawling_24GHz = crawling_24GHz[:min_length,:,:,:]
crawling_Xethru = crawling_Xethru[:min_length,:,:,:]

min_length = min(label_kneeling_77GHz.shape[0],label_kneeling_24GHz.shape[0],label_kneeling_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_kneeling_77GHz[:min_length]),axis=0)  
kneeling_77GHz = kneeling_77GHz[:min_length,:,:,:]
kneeling_24GHz = kneeling_24GHz[:min_length,:,:,:]
kneeling_Xethru = kneeling_Xethru[:min_length,:,:,:]

min_length = min(label_limping_with_RL_Stiff_77GHz.shape[0],label_limping_with_RL_Stiff_24GHz.shape[0],label_limping_with_RL_Stiff_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_limping_with_RL_Stiff_77GHz[:min_length]),axis=0)  
limping_with_RL_Stiff_77GHz = limping_with_RL_Stiff_77GHz[:min_length,:,:,:]
limping_with_RL_Stiff_24GHz = limping_with_RL_Stiff_24GHz[:min_length,:,:,:]
limping_with_RL_Stiff_Xethru = limping_with_RL_Stiff_Xethru[:min_length,:,:,:]

min_length = min(label_picking_up_an_object_77GHz.shape[0],label_picking_up_an_object_24GHz.shape[0],label_picking_up_an_object_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_picking_up_an_object_77GHz[:min_length]),axis=0)  
picking_up_an_object_77GHz = picking_up_an_object_77GHz[:min_length,:,:,:]
picking_up_an_object_24GHz = picking_up_an_object_24GHz[:min_length,:,:,:]
picking_up_an_object_Xethru = picking_up_an_object_Xethru[:min_length,:,:,:]

min_length = min(label_scissors_gait_77GHz.shape[0],label_scissors_gait_24GHz.shape[0],label_scissors_gait_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_scissors_gait_77GHz[:min_length]),axis=0)  
scissors_gait_77GHz = scissors_gait_77GHz[:min_length,:,:,:]
scissors_gait_24GHz = scissors_gait_24GHz[:min_length,:,:,:]
scissors_gait_Xethru = scissors_gait_Xethru[:min_length,:,:,:]

min_length = min(label_short_steps_77GHz.shape[0],label_short_steps_24GHz.shape[0],label_short_steps_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_short_steps_77GHz[:min_length]),axis=0)  
short_steps_77GHz = short_steps_77GHz[:min_length,:,:,:]
short_steps_24GHz = short_steps_24GHz[:min_length,:,:,:]
short_steps_Xethru = short_steps_Xethru[:min_length,:,:,:]

min_length = min(label_sitting_77GHz.shape[0],label_sitting_24GHz.shape[0],label_sitting_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_sitting_77GHz[:min_length]),axis=0)  
sitting_77GHz = sitting_77GHz[:min_length,:,:,:]
sitting_24GHz = sitting_24GHz[:min_length,:,:,:]
sitting_Xethru = sitting_Xethru[:min_length,:,:,:]

min_length = min(label_walking_away_from_Radar_77GHz.shape[0],label_walking_away_from_Radar_24GHz.shape[0],label_walking_away_from_Radar_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_walking_away_from_Radar_77GHz[:min_length]),axis=0)  
walking_away_from_Radar_77GHz = walking_away_from_Radar_77GHz[:min_length,:,:,:]
walking_away_from_Radar_24GHz = walking_away_from_Radar_24GHz[:min_length,:,:,:]
walking_away_from_Radar_Xethru = walking_away_from_Radar_Xethru[:min_length,:,:,:]

min_length = min(label_Walking_on_both_toes_77GHz.shape[0],label_Walking_on_both_toes_24GHz.shape[0],label_Walking_on_both_toes_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_Walking_on_both_toes_77GHz[:min_length]),axis=0)  
Walking_on_both_toes_77GHz = Walking_on_both_toes_77GHz[:min_length,:,:,:]
Walking_on_both_toes_24GHz = Walking_on_both_toes_24GHz[:min_length,:,:,:]
Walking_on_both_toes_Xethru = Walking_on_both_toes_Xethru[:min_length,:,:,:]

min_length = min(label_Walking_towards_radar_77GHz.shape[0],label_Walking_towards_radar_24GHz.shape[0],label_Walking_towards_radar_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_Walking_towards_radar_77GHz[:min_length]),axis=0)  
Walking_towards_radar_77GHz = Walking_towards_radar_77GHz[:min_length,:,:,:]
Walking_towards_radar_24GHz = Walking_towards_radar_24GHz[:min_length,:,:,:]
Walking_towards_radar_Xethru = Walking_towards_radar_Xethru[:min_length,:,:,:]


In [8]:
# 77 GHz
spectrogram_77GHz_concat = np.concatenate((bending_77GHz,crawling_77GHz),axis=0)
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,kneeling_77GHz               ),axis=0)
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,limping_with_RL_Stiff_77GHz  ),axis=0)
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,picking_up_an_object_77GHz   ),axis=0)
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,scissors_gait_77GHz          ),axis=0)
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,short_steps_77GHz            ),axis=0)
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,sitting_77GHz                ),axis=0)
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,walking_away_from_Radar_77GHz),axis=0)
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,Walking_on_both_toes_77GHz   ),axis=0)
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,Walking_towards_radar_77GHz  ),axis=0)
del bending_77GHz
del crawling_77GHz
del kneeling_77GHz               
del limping_with_RL_Stiff_77GHz  
del picking_up_an_object_77GHz   
del scissors_gait_77GHz          
del short_steps_77GHz            
del sitting_77GHz                
del walking_away_from_Radar_77GHz
del Walking_on_both_toes_77GHz   
del Walking_towards_radar_77GHz  
# 24 GHz
spectrogram_24GHz_concat = np.concatenate((bending_24GHz,crawling_24GHz),axis=0)
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,kneeling_24GHz               ),axis=0)
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,limping_with_RL_Stiff_24GHz  ),axis=0)
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,picking_up_an_object_24GHz   ),axis=0)
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,scissors_gait_24GHz          ),axis=0)
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,short_steps_24GHz            ),axis=0)
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,sitting_24GHz                ),axis=0)
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,walking_away_from_Radar_24GHz),axis=0)
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,Walking_on_both_toes_24GHz   ),axis=0)
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,Walking_towards_radar_24GHz  ),axis=0)
del crawling_24GHz
del bending_24GHz
del kneeling_24GHz               
del limping_with_RL_Stiff_24GHz  
del picking_up_an_object_24GHz   
del scissors_gait_24GHz          
del short_steps_24GHz            
del sitting_24GHz                
del walking_away_from_Radar_24GHz
del Walking_on_both_toes_24GHz   
del Walking_towards_radar_24GHz  

# Xethru
spectrogram_Xethru_concat = np.concatenate((bending_Xethru,crawling_Xethru),axis=0)
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,kneeling_Xethru               ),axis=0)
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,limping_with_RL_Stiff_Xethru  ),axis=0)
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,picking_up_an_object_Xethru   ),axis=0)
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,scissors_gait_Xethru          ),axis=0)
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,short_steps_Xethru            ),axis=0)
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,sitting_Xethru                ),axis=0)
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,walking_away_from_Radar_Xethru),axis=0)
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,Walking_on_both_toes_Xethru   ),axis=0)
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,Walking_towards_radar_Xethru  ),axis=0)

del crawling_Xethru
del bending_Xethru
del kneeling_Xethru               
del limping_with_RL_Stiff_Xethru  
del picking_up_an_object_Xethru   
del scissors_gait_Xethru          
del short_steps_Xethru            
del sitting_Xethru                
del walking_away_from_Radar_Xethru
del Walking_on_both_toes_Xethru   
del Walking_towards_radar_Xethru  





In [9]:
# One-Hot Encode Label
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(label_concat)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded_labels = onehot_encoder.fit_transform(integer_encoded)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
# Concat range-doppler data
range_doppler_concat = spectrogram_77GHz_concat
range_doppler_concat_label = label_concat
del spectrogram_77GHz_concat
del label_concat
# Shuffle concat range doppler
shuffle_indx = random.sample(range(0, range_doppler_concat.shape[0]), range_doppler_concat.shape[0]) # split validation data
range_doppler_concat_shuffle = range_doppler_concat[shuffle_indx,:,:,:]
range_doppler_concat_label_shuffle = range_doppler_concat_label[shuffle_indx,:]
del range_doppler_concat
# Concat range-doppler data
spectrogram_concat = spectrogram_24GHz_concat
spectrogram_concat_label = onehot_encoded_labels
del spectrogram_24GHz_concat
del onehot_encoded_labels
# Shuffle concat range doppler
spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]
del spectrogram_concat
# Third Spectrogram
third_spectrogram_concat = spectrogram_Xethru_concat
third_spectrogram_concat_shuffle = third_spectrogram_concat[shuffle_indx,:,:,:]
del spectrogram_Xethru_concat
del third_spectrogram_concat

In [11]:
# ---------------- Augmente and shuffle (train and test) data data ----------------
def mixup_augmentation(images,range_doppler_training_data,third_spectrogram_training_data, labels, repeat_of_mixup, alpha, beta):
    batch_size = images.shape[0]
    concat_images = np.zeros((batch_size*(repeat_of_mixup+1),images.shape[1],images.shape[2],images.shape[3]))
    concat_images_range_doppler = np.zeros((batch_size*(repeat_of_mixup+1),images.shape[1],images.shape[2],images.shape[3]))
    concat_images_third_spectrogram = np.zeros((batch_size*(repeat_of_mixup+1),images.shape[1],images.shape[2],images.shape[3]))
    concat_label = np.zeros((batch_size*(repeat_of_mixup+1),labels.shape[1]))
    
    for ii in range(repeat_of_mixup):
      # shuffle train dataset
      shuffle_indx_1 = random.sample(range(0, images.shape[0]), images.shape[0]) # split validation data
      images_shuffled_1 = images[shuffle_indx_1,:,:,:]
      range_doppler_training_data_shuffled_1 = range_doppler_training_data[shuffle_indx_1,:,:,:]
      third_spectrogram_training_data_shuffled_1 = third_spectrogram_training_data[shuffle_indx_1,:,:,:]
      labels_shuffled_1 = labels[shuffle_indx_1,:]

      shuffle_indx_2 = random.sample(range(0, images.shape[0]), images.shape[0]) # split validation data
      images_shuffled_2 = images[shuffle_indx_2,:,:,:]
      range_doppler_training_data_shuffled_2 = range_doppler_training_data[shuffle_indx_2,:,:,:]
      third_spectrogram_training_data_shuffled_2 = third_spectrogram_training_data[shuffle_indx_2,:,:,:]
      labels_shuffled_2 = labels[shuffle_indx_2,:]

      shuffle_indx_3 = random.sample(range(0, images.shape[0]), images.shape[0]) # split validation data
      images_shuffled_3 = images[shuffle_indx_3,:,:,:]
      range_doppler_training_data_shuffled_3 = range_doppler_training_data[shuffle_indx_3,:,:,:]
      third_spectrogram_training_data_shuffled_3 = third_spectrogram_training_data[shuffle_indx_3,:,:,:]
      labels_shuffled_3 = labels[shuffle_indx_3,:]

      shuffle_indx_4 = random.sample(range(0, images.shape[0]), images.shape[0]) # split validation data
      images_shuffled_4 = images[shuffle_indx_4,:,:,:]
      range_doppler_training_data_shuffled_4 = range_doppler_training_data[shuffle_indx_4,:,:,:]
      third_spectrogram_training_data_shuffled_4 = third_spectrogram_training_data[shuffle_indx_4,:,:,:]
      labels_shuffled_4 = labels[shuffle_indx_4,:]

      # Sample lambda and reshape it to do the mixup
      # ll = np.random.beta(alpha, beta, (batch_size,1,1,1))
      gaussian_mean = 0.11
      gaussian_std = 0.02
      ll = np.random.normal(gaussian_mean, gaussian_std, (batch_size,1,1,1))
      x_l = np.reshape(ll, (batch_size,1,1,1))
      y_l = np.reshape(ll, (batch_size,1))
    
      ll_2 = np.random.normal(gaussian_mean, gaussian_std, (batch_size,1,1,1))
      x_l_2 = np.reshape(ll_2, (batch_size,1,1,1))
      y_l_2 = np.reshape(ll_2, (batch_size,1))

      ll_3 = np.random.normal(gaussian_mean, gaussian_std, (batch_size,1,1,1))
      x_l_3 = np.reshape(ll_3, (batch_size,1,1,1))
      y_l_3 = np.reshape(ll_3, (batch_size,1))

      
      # Perform mixup on both images and labels by combining a pair of images/labels
      images_mixup = images_shuffled_1 * x_l + images_shuffled_2 * x_l_2 + images_shuffled_3 * x_l_3 + images_shuffled_4 * (1 - x_l - x_l_2 - x_l_3)
      images_mixup_range_doppler = range_doppler_training_data_shuffled_1 * x_l + range_doppler_training_data_shuffled_2 * x_l_2 +\
       range_doppler_training_data_shuffled_3 * x_l_3 + range_doppler_training_data_shuffled_4 * (1 - x_l - x_l_2 - x_l_3)
      images_mixup_third_spectrogram = third_spectrogram_training_data_shuffled_1 * x_l + third_spectrogram_training_data_shuffled_2 * x_l_2 +\
       third_spectrogram_training_data_shuffled_3 * x_l_3 + third_spectrogram_training_data_shuffled_4 * (1 - x_l - x_l_2 - x_l_3)
      labels_mixup = labels_shuffled_1 * y_l + labels_shuffled_2 * y_l_2  + labels_shuffled_3 * y_l_3 + labels_shuffled_4 * (1 - y_l - y_l_2 - y_l_3)

      # images_mixup = images_shuffled_1 * x_l + images_shuffled_2 * (1 - x_l)
      # images_mixup_range_doppler = range_doppler_training_data_shuffled_1 * x_l + range_doppler_training_data_shuffled_2 * (1 - x_l)
      # images_mixup_third_spectrogram = third_spectrogram_training_data_shuffled_1 * x_l + third_spectrogram_training_data_shuffled_2 * (1 - x_l)
      # labels_mixup = labels_shuffled_1 * y_l + labels_shuffled_2 * (1 - y_l)

      concat_images[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup
      concat_images_range_doppler[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup_range_doppler
      concat_images_third_spectrogram[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup_third_spectrogram
      concat_label[ii*batch_size:(ii+1)*batch_size,:] = labels_mixup

    concat_images[repeat_of_mixup*batch_size:,:,:,:] = images
    concat_images_range_doppler[repeat_of_mixup*batch_size:,:,:,:] = range_doppler_training_data
    concat_images_third_spectrogram[repeat_of_mixup*batch_size:,:,:,:] = third_spectrogram_training_data
    concat_label[repeat_of_mixup*batch_size:,:] = labels
    return (concat_images,concat_images_range_doppler,concat_images_third_spectrogram, concat_label)

def split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,third_spectrogram,range_doppler_concat_label_shuffle_train,\
                                       repeat_of_mixup, augmentation_enable):
  # ---------------- Parameters ----------------
  # to split train and validation with same distribution
  alpha = 0.1
  beta = 5
  dummy_label = np.zeros((spectrogram_concat_shuffle_train.shape[0],1))
  for randomlist_for_train_indx, randomlist_for_validation_indx in kfold.split(spectrogram_concat_shuffle_train,dummy_label):   
    randomlist_for_validation_indx
  # size_of_validation = int(0.2*spectrogram_concat_shuffle_train.shape[0])
  # # Split validation
  # randomlist_for_validation_indx = random.sample(range(0, range_doppler_concat_shuffle_train.shape[0]), size_of_validation) # split validation data
  # randomlist_for_train_indx = np.delete(range(0, range_doppler_concat_shuffle_train.shape[0]), randomlist_for_validation_indx) # split training data
  # get validation data
  third_spectrogram_validation_data = third_spectrogram[randomlist_for_validation_indx,:,:,:]
  spectrogram_validation_data = spectrogram_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  spectrogram_validation_labels = range_doppler_concat_label_shuffle_train[randomlist_for_validation_indx,:]
  range_doppler_validation_data = range_doppler_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  # get training data
  third_spectrogram_training_data = third_spectrogram[randomlist_for_train_indx,:,:,:]
  spectrogram_training_data = spectrogram_concat_shuffle_train[randomlist_for_train_indx,:,:,:]
  spectrogram_training_labels = spectrogram_concat_label_shuffle_train[randomlist_for_train_indx,:]
  range_doppler_training_data = range_doppler_concat_shuffle_train[randomlist_for_train_indx,:,:,:]

  (spectrogram_augmented_image,range_doppler_augmented_image,third_spectrogram_augmented_image,spectrograms_label)=\
   mixup_augmentation(spectrogram_training_data,range_doppler_training_data,third_spectrogram_training_data, spectrogram_training_labels, repeat_of_mixup, alpha, beta)

  return (spectrogram_augmented_image,range_doppler_augmented_image,third_spectrogram_augmented_image,spectrograms_label,\
     spectrogram_validation_data,range_doppler_validation_data,third_spectrogram_validation_data, spectrogram_validation_labels)

def normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle)
    range_doppler_concat_shuffle = layer(range_doppler_concat_shuffle)
  else:
    range_doppler_concat_shuffle = range_doppler_concat_shuffle
  return(range_doppler_concat_shuffle)


In [12]:
range_doppler_concat_shuffle_new = np.zeros((range_doppler_concat_shuffle.shape[0],128,128,1))
for ii in range(range_doppler_concat_shuffle.shape[0]):
  aa = range_doppler_concat_shuffle[ii,:,:,0]
  range_doppler_concat_shuffle_new[ii,:,:,0] = cv2.resize(aa, (range_doppler_concat_shuffle_new.shape[2], range_doppler_concat_shuffle_new.shape[1]),interpolation = cv2.INTER_CUBIC)
range_doppler_concat_shuffle = range_doppler_concat_shuffle_new
del aa
del range_doppler_concat_shuffle_new

spectrogram_concat_shuffle_new = np.zeros((spectrogram_concat_shuffle.shape[0],128,128,1))
for ii in range(spectrogram_concat_shuffle.shape[0]):
  aa = spectrogram_concat_shuffle[ii,:,:,0]
  spectrogram_concat_shuffle_new[ii,:,:,0] = cv2.resize(aa, (spectrogram_concat_shuffle_new.shape[2], spectrogram_concat_shuffle_new.shape[1]),interpolation = cv2.INTER_CUBIC)
spectrogram_concat_shuffle = spectrogram_concat_shuffle_new
del aa
del spectrogram_concat_shuffle_new

third_spectrogram_concat_shuffle_new = np.zeros((third_spectrogram_concat_shuffle.shape[0],128,128,1))
for ii in range(third_spectrogram_concat_shuffle.shape[0]):
  aa = third_spectrogram_concat_shuffle[ii,:,:,0]
  third_spectrogram_concat_shuffle_new[ii,:,:,0] = cv2.resize(aa, (third_spectrogram_concat_shuffle_new.shape[2], third_spectrogram_concat_shuffle_new.shape[1]),interpolation = cv2.INTER_CUBIC)
third_spectrogram_concat_shuffle = third_spectrogram_concat_shuffle_new
del aa
del third_spectrogram_concat_shuffle_new

In [13]:
normalize_inputs_enable = 1
range_doppler_concat_shuffle = normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable)
spectrogram_concat_shuffle = normalize_inputs(spectrogram_concat_shuffle, normalize_inputs_enable)
third_spectrogram_concat_shuffle = normalize_inputs(third_spectrogram_concat_shuffle, normalize_inputs_enable)
range_doppler_concat_shuffle = np.float32(range_doppler_concat_shuffle)
spectrogram_concat_shuffle = np.float32(spectrogram_concat_shuffle)
third_spectrogram_concat_shuffle = np.float32(third_spectrogram_concat_shuffle)


In [16]:

# ---------- Parameters ----------------
augmentation_enable = True
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
epoch_number = 100
batch_size = 32
dense_size = 128
dropout_prob_cnn = 0.1
dropout_prob_dense = 0.4
repeat_of_mixup = 2
number_of_repeat = 5
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  f1_score_per_fold = []
  for train, test in kfold.split(range_doppler_concat_shuffle,range_doppler_concat_label_shuffle):   
    gc.collect()
    K.clear_session()

    randomlist_for_test_indx = test
    randomlist_for_train_indx = train
    # test data
    spectrogram_concat_shuffle_test = spectrogram_concat_shuffle[randomlist_for_test_indx,:,:,:]
    spectrogram_concat_label_shuffle_test = spectrogram_concat_label_shuffle[randomlist_for_test_indx,:]
    range_doppler_concat_shuffle_test = range_doppler_concat_shuffle[randomlist_for_test_indx,:,:,:]
    third_spectrogram_concat_shuffle_test = third_spectrogram_concat_shuffle[randomlist_for_test_indx,:,:,:]
    #train data
    spectrogram_concat_shuffle_train = spectrogram_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]
    range_doppler_concat_shuffle_train = range_doppler_concat_shuffle[randomlist_for_train_indx,:,:,:]
    third_spectrogram_concat_shuffle_train = third_spectrogram_concat_shuffle[randomlist_for_train_indx,:,:,:]
    # ---------------- Split labels to equal them during augmentation for Validation ----------------
    (spectrogram_augmented_image,range_doppler_augmented_image,third_spectrogram_augmented_image,spectrogram_concat_label_shuffle_concat,\
     validation_spectrogram,validation_range_doppler,validation_third_spectrogram, spectrogram_validation_labels)  =\
      split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,third_spectrogram_concat_shuffle_train,\
                                         spectrogram_concat_label_shuffle_train,\
                                         repeat_of_mixup, augmentation_enable)

    def encoder_for_range_doppler(input_shape):
      input = Input(shape=input_shape)
      x = Conv2D(4, (3, 3),padding='same',kernel_initializer="glorot_normal")(input)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(8, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(32, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(0.3)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Flatten()(x)
      x = Dense(128)(x)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(dropout_prob_dense)(x)
      return Model(input, x)

    def encoder_for_spectrogram(input_shape):
      input = Input(shape=input_shape)
      x = Conv2D(4, (3, 3),padding='same',kernel_initializer="glorot_normal")(input)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(8, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(32, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(0.3)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Flatten()(x)
      x = Dense(128)(x)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(dropout_prob_dense)(x)
      return Model(input, x)

    def encoder_for_third_spectrogram(input_shape):
      input = Input(shape=input_shape)
      x = Conv2D(4, (3, 3),padding='same',kernel_initializer="glorot_normal")(input)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(8, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(32, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(0.3)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Flatten()(x)
      x = Dense(128)(x)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(dropout_prob_dense)(x)
      return Model(input, x)


    def decoder_for_concat(input_shape):
      input = Input(shape=input_shape)
      x = Dense(256)(input)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(dropout_prob_dense)(x)
      x = Dense(dense_size)(x)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(0.3)(x)
      x = Dense(11, activation="softmax")(x)
      return Model(input, x)
    input_shape = range_doppler_concat_shuffle.shape[1:]
    base_network_range_doppler = encoder_for_range_doppler(input_shape)
    range_doppler_input  = Input(shape=input_shape)
    processed_range_doppler  = base_network_range_doppler(range_doppler_input)

    input_shape = spectrogram_concat_shuffle.shape[1:]
    base_network_spectrogram = encoder_for_spectrogram(input_shape) 
    spectrogram_input  = Input(shape=input_shape)
    processed_spectrogram  = base_network_spectrogram(spectrogram_input)

    input_shape = spectrogram_concat_shuffle.shape[1:]
    base_network_spectrogram = encoder_for_third_spectrogram(input_shape) 
    third_spectrogram_input  = Input(shape=input_shape)
    processed_third_spectrogram  = base_network_spectrogram(third_spectrogram_input)

    concat_layer = Concatenate()([processed_range_doppler, processed_spectrogram, processed_third_spectrogram])

    base_decoder_network = decoder_for_concat((concat_layer.shape[1]))
    out = base_decoder_network(concat_layer)

    model = Model(inputs=[range_doppler_input, spectrogram_input, third_spectrogram_input], outputs=[out])

   
    print(model.summary())
    # ---------------- Compile and Fit ----------------
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    earlyStopping = EarlyStopping(monitor='val_loss', patience=35, verbose=0,restore_best_weights=True, mode='min')
    # earlyStopping = EarlyStopping(monitor='val_accuracy', patience=15, verbose=0,restore_best_weights=True, mode='max')
    t = time.time()
    history = model.fit((range_doppler_augmented_image,spectrogram_augmented_image,third_spectrogram_augmented_image),(spectrogram_concat_label_shuffle_concat),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    shuffle = True,
                    verbose=0,
                    callbacks=[earlyStopping],
                    validation_data = ((validation_range_doppler,validation_spectrogram,validation_third_spectrogram) , (spectrogram_validation_labels)))
    elapsed = time.time() - t
    tf.keras.models.load_model
    test_loss, test_accuracy  = model.evaluate([range_doppler_concat_shuffle_test,spectrogram_concat_shuffle_test,third_spectrogram_concat_shuffle_test],\
                                               [spectrogram_concat_label_shuffle_test],
                  batch_size=batch_size)
    # ---------------- Get Test Results ----------------
    y_test_predicted = model.predict((range_doppler_concat_shuffle_test,spectrogram_concat_shuffle_test,third_spectrogram_concat_shuffle_test), batch_size=batch_size)
    # ----- Binarize y_test_predicted values -----
    y_test_predicted_binary = np.zeros(y_test_predicted.shape[0])
    for ii in range(y_test_predicted.shape[0]):
      if argmax(y_test_predicted[ii,:]) == 0:
        y_test_predicted_binary[ii] = 0
      elif argmax(y_test_predicted[ii,:]) == 1:
        y_test_predicted_binary[ii] = 1
      else:
        y_test_predicted_binary[ii] = 2
    range_doppler_concat_label_shuffle_test1 = range_doppler_concat_label_shuffle[randomlist_for_test_indx,:]
    test_precision, test_recall, test_f1_score, support = precision_recall_fscore_support(range_doppler_concat_label_shuffle_test1, y_test_predicted_binary, average='macro')
    # ----------------------------------------------------------------------------------------------------------
    test_accuracy_per_fold.append(test_accuracy)
    f1_score_per_fold.append(test_f1_score)
  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  f1_score_per_run.append(sum(f1_score_per_fold)/num_folds)
  print(test_accuracy_per_run)
  print(f1_score_per_run)

print(f'Mean test accuracy is {"{:.2f}".format(sum(test_accuracy_per_run)/number_of_repeat)}, mean test f1 score is {"{:.2f}".format(sum(f1_score_per_run)/number_of_repeat)}, \
max test accuracy is {"{:.2f}".format(max(test_accuracy_per_run))}, max test f1 score is {"{:.2f}".format(max(f1_score_per_run))}, \
min test accuracy is {"{:.2f}".format(min(test_accuracy_per_run))}, min test f1 score is {"{:.2f}".format(min(f1_score_per_run))}, \
std of test accuracy is {"{:.2f}".format(np.std(test_accuracy_per_run, axis=0))}, std of test f1 score is {"{:.2f}".format(np.std(f1_score_per_run, axis=0))}')

print(f'Time elapsed through all process: {"{:.2f}".format(elapsed)}, sec')

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_4 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_6 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                          

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_4 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_6 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                          

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_4 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_6 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                          

...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_4 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_6 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                          

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_4 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_6 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                          

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
t_test = time.time()
y_test_predicted = model.predict((range_doppler_concat_shuffle_test[0:1,:,:,:],spectrogram_concat_shuffle_test[0:1,:,:,:],third_spectrogram_concat_shuffle_test[0:1,:,:,:]))
test_elapsed = time.time() - t_test 
print(f'Time elapsed during test time: {"{:.2f}".format(test_elapsed)}, sec')

Time elapsed during test time: 0.27, sec


In [19]:
print(f'Min test accuracy is {"{:.2f}".format(min(test_accuracy_per_run))}, \
Max test accuracy is {"{:.2f}".format(max(test_accuracy_per_run))}, \
Mean test accuracy is {"{:.2f}".format(sum(test_accuracy_per_run)/number_of_repeat)}, \
std of test accuracy is {"{:.2f}".format(np.std(test_accuracy_per_run, axis=0))}.')
print(f'Time elapsed during training time: {"{:.2f}".format(elapsed)}, sec')
print(f'Time elapsed during test time: {"{:.2f}".format(test_elapsed)}, sec')

Min test accuracy is 0.82, Max test accuracy is 0.84, Mean test accuracy is 0.83, std of test accuracy is 0.01.
Time elapsed during training time: 70.26, sec
Time elapsed during test time: 0.27, sec
